<img src="https://github.com/stanfordnlp/dspy/blob/main/docs/images/DSPy8.png?raw=1" alt="DSPy7 图片" height="150"/>

### **SolveGSM8k**: 使用 DSPy 解决小学数学问题

<a target="_blank" href="https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/examples/math/gsm8k/gsm8k_assertions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/>
</a>

这份笔记构建在DSPy框架的基础概念之上。DSPy提供了一种新颖的以编程为中心的方法来利用语言和检索模型。它提供了一种独特的提示、推理、微调和工具增强的混合，所有这些都封装在一个简约的Python语法下。

我们将重点关注三个部分：

1. 定义一个DSPy程序并评估其性能。
3. 使用运行时的DSPy断言和建议来约束DSPy程序的行为。
2. 通过上下文学习和提示调整来优化DSPy程序。

In [ ]:
# 设置你的OpenAI API密钥
import os
os.environ["OPENAI_API_KEY"]="在这里粘贴你的密钥"

### 步骤 -1. **安装 Cache 和 DSPy**（运行折叠的单元格）

默认情况下，单元格是折叠的。运行以下单元格将设置缓存并安装所有依赖项和 DSPy。

第一个单元格确保此笔记本中的所有后续 LM 调用都将使用缓存的 OpenAI API 结果。删除此步骤可能会显著增加所有后续 DSPy 程序的运行时间，具体取决于您的 OpenAI 账户设置。

In [ ]:
# 这个单元格设置缓存（预先计算的OpenAI调用结果）。
!rm -r gsm8k_cache || true
!rm -r dspy || true
!git clone https://github.com/Shangyint/gsm8k_cache.git

import os
repo_clone_path = '/content/gsm8k_cache'

# 检查'/content'是否可写
if not os.access('/content', os.W_OK):
    # 如果'/content'不可写，选择一个替代目录
    # 例如：使用相对于当前工作目录的目录
    repo_clone_path = os.path.join(os.getcwd(), 'gsm8k_cache')

# 为这个笔记本设置缓存
os.environ["DSP_NOTEBOOK_CACHEDIR"] = repo_clone_path

在我们开始之前，我们需要安装 DSPy 和所有依赖项。

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import regex as re
import json

import pkg_resources # 如果未安装该包，则安装该包
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install git+https://github.com/stanfordnlp/dspy.git
    !pip install openai~=0.28.1

from rich import print
import dspy

from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

### 步骤 0. **开始**（运行已折叠的单元格）

我们将从导入我们的数据集 GSM8K 开始，这是一个包含 8.5K 由人类问题编写者创建的高质量语言多样的小学数学问题数据集。我们对数据集进行了预先洗牌，并将其分成三个较小的集合 - 训练集、开发集（验证集）和测试集。为简单起见，我们将使用训练集和开发集。

如果您想检查数据集并查看我们如何设置 DSPy 来使用 OpenAI gpt3，请展开此步骤。

In [ ]:
gms8k = GSM8K()  # 创建一个GSM8K对象
trainset, devset = gms8k.train, gms8k.dev  # 将训练集和验证集分别赋值给trainset和devset
len(trainset), len(devset)  # 输出训练集和验证集的长度

现在我们可以检查一些示例。

In [ ]:
# 选择第11个样本作为数学问题示例
math_problem_example = devset[10]

# 打印问题
print(f"Question: {math_problem_example.question}\n")

# 打印正确推理过程
print(f"Gold Reasoning: {math_problem_example.gold_reasoning}\n")

# 打印答案
print(f"Answer: {math_problem_example.answer}")

然后我们设置语言模型（LM）。**DSPy**支持多个API和本地模型。在这个笔记本中，我们将使用GPT-3.5（`gpt-3.5-turbo`）。

我们将**DSPy**配置为默认使用turbo LM（`gpt-3.5-turbo`）。如果需要，这可以被覆盖以用于程序的本地部分。

In [ ]:

# 创建一个OpenAI对象，使用gpt-3.5-turbo模型，设置最大生成标记数为500
turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=500)

# 配置dspy库的设置，将语言模型设置为turbo
dspy.settings.configure(lm=turbo)

### 步骤 1. **第一个 DSPy 程序**

在 **DSPy** 中，我们将保持**以声明方式定义模块**和**在管道中调用它们来解决任务**之间的清晰分离。

这使您可以专注于管道的信息流。**DSPy** 将自动优化**如何提示**（或微调）LMs **以适应您特定的管道**，使其运行良好。

如果您有 PyTorch 的经验，您可以将 DSPy 视为基础模型空间的 PyTorch。在我们看到它实际运行之前，让我们首先了解一些关键部分。

##### 使用语言模型：**签名** 和 **预测器**

在 **DSPy** 程序中每次调用 LM 都需要一个 **签名**。

一个签名由三个简单元素组成：

- LM 应该解决的子任务的最小描述。
- 我们将提供给 LM 的一个或多个输入字段的描述（例如，输入问题）。
- 我们期望从 LM 获得的一个或多个输出字段的描述（例如，问题的答案）。

让我们为基本数学问题解决定义一个简单的签名。

In [ ]:
class SimpleMathSignature(dspy.Signature):
    """回答数学问题。"""

    # 输入字段，描述为一个简单的数学问题
    question = dspy.InputField(desc="A simple math question.")
    
    # 输出字段，描述为数学问题的答案
    answer = dspy.OutputField(desc="The answer to the math question.")

在`SimpleMathSignature`中，文档字符串描述了这里的子任务（即回答数学问题）。每个`InputField`或`OutputField`也可以选择包含一个描述`desc`。当没有提供时，它会从字段的名称中推断（例如，`question`）。

请注意，在**DSPy**中，这个签名并没有什么特别之处。我们可以很容易地定义一个签名，它接收来自PDF的长片段并输出结构化信息，例如。

DSPy签名的一个技巧是，当它只包含执行简单任务的简单字段时，我们可以用一种语法糖`question -> answer`来替换整个类定义。现在，让我们用DSPy预测器定义我们的第一个DSPy程序。预测器是一个模块，它知道如何使用LM来实现一个签名。重要的是，预测器可以学习适应任务的行为！

In [ ]:
basic_math_solver = dspy.Predict("question -> answer") # Alternatively, we can write dspy.Predict(SimpleMathSignature)

**`DSPy.Predict`** 是最简单的 DSPy 预测器。现在，我们可以用一个手工制作的问题调用这个最小的 _程序_：

In [ ]:
# 使用 basic_math_solver 函数计算问题 "What is 1+1+1?" 的答案
prediction = basic_math_solver(question="What is 1+1+1?")

# 打印计算结果
print(f"Answer: {prediction.answer}")

在上面的示例中，我们向预测器提出了一个简单的数学问题“1+1+1等于多少？”。模型输出了一个答案（“3”）。

为了查看清晰，我们可以检查这个极其基本的预测器是如何实现我们的签名的。让我们检查我们的LM（**turbo**）的历史。

In [ ]:
# 使用 turbo.inspect_history 函数检查历史记录，n=1 表示只检查最近的一条历史记录
_ = turbo.inspect_history(n=1)

好的。现在让我们定义实际的程序。这是一个从`dspy.Module`继承的类。

它需要两个方法：

- `__init__`方法将简单地声明它需要的子模块：这次，我们将使用一个更复杂的预测器，它实现了Chain-of-Thought提示`dspy.ChainOfThought`。`dspy.ChainOfThought`将添加另一个名为"rationale"的字段作为输出，以帮助模型逐步思考。
- `forward`方法将描述使用我们拥有的模块来回答问题的控制流程（这里，我们只有一个模块）。

In [ ]:
# 导入必要的模块
import dspy

# 定义一个简单的数学求解器类
class SimpleMathSolver(dspy.Module):
    def __init__(self):
        # 创建一个思维链
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        # 通过思维链进行推理
        pred = self.prog(question=question)
        return pred

# 创建一个SimpleMathSolver的实例
simple_math_solver = SimpleMathSolver()

#### **练习**
创建你自己的数学问题，使用我们刚刚定义的数学求解器。然后，使用 `turbo.inspect_history` 检查 LM 的跟踪，看看与 `dspy.Predict` 预测器相比有哪些变化。

In [ ]:
### Fill this code cell

我们现在可以在验证集上评估我们的简单数学求解器。

首先，让我们评估预测答案的准确性。我们提供了一个简单的度量函数，名为`gsm8k_metric`，它主要从模型输入中提取数值答案。

In [ ]:
# 创建一个Evaluate对象，用于评估模型在开发集上的性能
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=16, display_progress=True, display_table=5)

# 使用evaluate对象评估simple_math_solver模型的性能
evaluate(simple_math_solver)

### 步骤 2. **使用 DSPy 断言添加约束**

我们在验证集上获得了**61.67%**的准确率，还不错！但我们也注意到了两件事情：1）很多答案是句子，而不是我们想要的数值结果。虽然我们能够在`gsm8k_metric`中解析大多数答案，但生成与答案无关的标记可能会对整体准确性产生负面影响；2）有些推理可能不包含所需的计算步骤，就像下面的例子一样。

In [ ]:
# 调用 simple_math_solver 函数处理 devset 列表中第一个元素的 question 字段
simple_math_solver(devset[0].question)

# 调用 turbo 模块的 inspect_history 函数，并将返回值赋给变量 _
_ = turbo.inspect_history()

幸运的是，在DSPy中，我们可以利用一个简单而强大的构造称为**LM Assertions**来约束LM的输出。例如，在这里，我们可以说：

```python
dspy.Suggest(len(pred.answer) < 10, "Your Answer should be a number.")
```

这个建议告诉DSPy运行时，我们期望数学求解器的答案很短，如果LM未能产生这样的答案，我们指示LM说"Your Answer should be a number."。

在DSPy中的LM断言可以是硬约束`Assert`，也可以是软约束`Suggest`。LM断言接受两个参数，一个是要测试的谓词，类似于传统断言；然后，我们还需要一个额外的"错误消息"来指导语言模型在失败时进行自我完善。

#### 具有建议功能的数学求解器
我们可以将我们得到的两个观察结果编码为两个建议，并将它们添加到`SimpleMathSolver`中：

In [ ]:
def extract_number(question):
    # 从问题中提取数字
    numbers = [int(s) for s in question.split() if s.isdigit()]
    return numbers

def has_numbers(rationale, numbers):
    # 判断理由中是否包含特定数字
    for number in numbers:
        if str(number) not in rationale:
            return False, number
    return True, None

class SimpleMathSolverWithSuggest(dspy.Module):
    def __init__(self):
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        pred = self.prog(question=question)
        rationale_has_numbers, missing_number = has_numbers(pred.rationale, extract_number(question))
        dspy.Suggest(rationale_has_numbers, f"Your Reasoning should contain {missing_number}.")
        dspy.Suggest(len(pred.answer) < 10, "Your Answer should be a number.")
        return pred

simple_math_solver_suggest = SimpleMathSolverWithSuggest().activate_assertions()

现在我们可以重新运行我们的数学求解器来解决第一个问题，并查看 DSPy 中的 LM 断言是如何在内部修复这些错误的。

In [ ]:
# 调用simple_math_solver_suggest函数，并传入devset[0].question作为参数
simple_math_solver_suggest(devset[0].question)

# 调用turbo.inspect_history函数，设置参数n=3，并将结果赋值给变量_
_ = turbo.inspect_history(n=3)

最后，让我们评估`simple_math_solver_suggest`的性能：

In [ ]:
# 调用evaluate函数，传入simple_math_solver_suggest作为参数
evaluate(simple_math_solver_suggest)

### 步骤 3. **使用优化器编译 DSPy 程序**

DSPy 的另一个很酷的功能是优化器！

DSPy 优化器是一种算法，可以调整 DSPy 程序的参数（即提示和/或 LM 权重），以最大化您指定的指标，如准确性。

DSPy 中有许多内置的优化器，应用着非常不同的策略。一个典型的 DSPy 优化器需要三样东西：

1. 您的 **DSPy 程序**。这可以是一个单模块（例如，dspy.Predict）或一个复杂的多模块程序。

2. 您的 **指标**。这是一个评估您的程序输出的函数，并为其分配一个分数（分数越高越好）。

3. 一些 **训练输入**。这可能非常少（即只有 5 或 10 个示例）且不完整（仅为您的程序输入，没有任何标签）。

如果您恰好有大量数据，DSPy 可以利用这些数据。但您可以从小处开始，获得强大的结果。

在本教程中，我们演示了一个名为`BootstrapFewShotWithRandomSearch`的DSPy优化器，它从训练集中引导演示并搜索最佳的演示组合。这里有两点需要注意：
1. 大多数优化器与LM断言一起工作。
2. 这一步骤需要大量的时间/计算资源。因此，我们缓存了API调用。好处是，一旦优化了程序，您可以保存编译后的DSPy程序，并在以后重复使用它！

In [ ]:
# 创建一个优化器对象，使用BootstrapFewShotWithRandomSearch算法
optimizer = BootstrapFewShotWithRandomSearch(gsm8k_metric, max_bootstrapped_demos=3, max_labeled_demos=6, num_candidate_programs=6)

# 编译简单数学求解器simple_math_solver，使用训练集trainset的全部数据和验证集devset的前100个数据
compiled_prog = optimizer.compile(student=simple_math_solver, trainset=trainset[:], valset=devset[:100])

# 编译简单数学求解器simple_math_solver_suggest，使用训练集trainset的全部数据和验证集devset的前100个数据
compiled_prog_suggest = optimizer.compile(student=simple_math_solver_suggest, trainset=trainset[:], valset=devset[:100])

In [ ]:
# 评估编译后的程序
evaluate(compiled_prog)

In [ ]:
# 使用建议评估编译后的程序
evaluate(compiled_prog_suggest)

现在我们检查一下之前的例子，看看优化器是如何调整提示的：

In [ ]:
# 编译并执行devset中第一个问题
compiled_prog(devset[0].question)

# 检查并打印Turbo的历史记录
_ = turbo.inspect_history()

### 更多DSPy教程

1. [DSPy简介](https://github.com/stanfordnlp/dspy/blob/main/intro.ipynb)
2. [DSPy断言](https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/examples/longformqa/longformqa_assertions.ipynb)
3. [测验生成](https://github.com/stanfordnlp/dspy/blob/main/examples/quiz/quiz_assertions.ipynb)
4. ... 更多请查看[DSPy github](https://github.com/stanfordnlp/dspy)

#### 联系人：Shangyin Tan (shangyin@berkeley.edu)